<h1>Отчет о проделанной работе</h1>

<p>В прошлой версии, реализованной в среде MatLab, не удалось корректно реализовать рассчет сферических функций Бесселя, поэтому я принял решение переписать весь код.</p>
<p>Данная программа реализована на языке программирования python.</p>

<p>Подключим необходимые библиотеки.</p>

In [145]:
from math import *
from scipy import special
import numpy as np
import cmath
import time
import plotly.graph_objects as go
from IPython.display import display, Image, SVG, Math, YouTubeVideo

<p>Определим функцию $sinc(x)$.</p>

In [146]:
def sinc(x):
    '''
    Compute the sinc-function: sin(x) / x
    '''
    try:  
        return sin(x) / x
    except ZeroDivisionError:
        return 1.

<h2>Поставим теперь задачу:</h2>
<p>Требуется написать такую функцию, результатом работы которой был бы кортеж из двух одномерных массивов, один из которых представлял бы из себя массив сферических функции Бесселя всех порядков от нуля до передаваемого в эту функцию параметра - последнего порядка, назовем его $N_{max}$, а другой, соответственно, массив сферических функций Неймана порядков от нуля до $N_{max}$.</p>
<p>Сферические функции Бесселя и Неймана и их производные могут быть рассчитаны по известным рекуррентным соотношениям:
</p>

$$s_n(\zeta) = \frac{2n - 1}{\zeta}s_{n-1}(\zeta)-s_{n-2}(\zeta) \tag{A1}$$

$$s_n^{'}(\zeta) = s_{n-1}(\zeta)-\frac{n + 1}{\zeta}s_{n}(\zeta) \tag{A2}$$

<p>где $s_n(\zeta)$ - условное обозначение одного из видов сферических функций, $\zeta$ - аргумент соответствующей функции.</p>
<p>Рекуррентная процедура $(А1)$ использует известные выражения для функций 0-го и 1-го порядков:</p>
$$j_0(\zeta)=\frac{sin\zeta}{\zeta},$$

$$j_1(\zeta)=-j_0^{'}(\zeta)=-\frac{cos\zeta}{\zeta}+\frac{sin\zeta}{\zeta^2},$$

$$y_0(\zeta)=-\frac{cos\zeta}{\zeta},$$

$$y_1(\zeta)=-y_0^{'}(\zeta)=-\frac{sin\zeta}{\zeta}-\frac{cos\zeta}{\zeta^2}.$$

<p>Но есть маленький нюанс. Процедура $(А1)$ является устойчивой для функции Неймана любого порядка и функций Бесселя порядка $n \leq \zeta$. Функции Бесселя порядков $n \geq \zeta$ находятся с помощью обратных рекуррентных соотношений:</p>

$$j_n(\zeta) = \frac{2n+3}{\zeta}j_{n+1}(\zeta)-j_{n+2}(\zeta) \tag{A3}$$

<p>Для запуска процедуры $(А3)$ с некоторого номера $N$ необходимо иметь значения функций $j_{N+2}(\zeta)$ и $j_{N+1}(\zeta)$, которые заранее неизвестны.  Однако рекуррентный алгоритм $(А3)$ довольно быстро выходит на ветвь, соответствующую функциям Бесселя, т.е. конкретные значения стартовых функций  и  не важны. Например, их можно выбрать равными 0 и 1 соответственно, а сам номер $N$ выбрать существенно, на несколько сотен, превышающим наибольшее из требуемых значений номера $n$. Рекуррентная процедура $(А3)$ при этом определяет функции Бесселя с точностью до некоторого неизвестного множителя: $\tilde{j_n}(\zeta) = \chi j_n(\zeta)$.</p>
<p>С уменьшением номера $n$ значения функций  чрезвычайно быстро нарастают и, вообще говоря, могут приблизиться к величине компьютерной бесконечности.  Поэтому целесообразно при некоторых значениях номера проводить их перенормировку.</p>

<h3>Реализация функции $\textit{upperRecurency}$</h3>
<p>Как было отмечено выше, на вход в эту функцию передается целочисленное число $N_{max}$ - тот порядок, до которого будем рассчитывать сферические функции и значение аргумента всех рассчитываемых функций - $\text{value}$.</p>
<p>Учтем, что порядки функций лежат в диапазоне от нуля до $N_{max}$, поэтому общее количество расчитываемых функций будет равно $N_{max} + 1$</p>
<p>Создадим три нулевых одномерных массива размерностью $N_{max} + 1$. Массив сферических функций Бесселя: $sphj$(spherical j), сферических функций Неймана: $sphy$(spherical y). В силу неустойчивости алгоритма $(А1)$, будем использовать массив $\textit{new_sphj}$ для новых сферических функций Бесселя, получаемых путем использования алгоритма $(А3)$.</p>
<p>Проверим на ненулевое значение аргумента, для избежания ошибки деления на ноль, при заполнении нулевых и первых порядков, которые известны нам точно. Если же, пользователь хочет получить все функции с аргументом равным нулю, то для инициализации первых и нулевых порядков обратимся к встроенным функциям, в которых это предусмотрено.</p>
<p>Используем процедуру $(А1)$ для всех сферических функций Неймана требуемых порядков, и для той части сферических функций Бесселя, чей порядок меньше либо равен значению аргумента. Результатом получаем полностью готовый массив для функций Неймана, и, в общем случае, неполностью заполненный массив функций Бесселя.</p>
<p>Для следующей части функций Бесселя, используем процедуру $(А3)$, для этого, согласно теории, описанной выше, уходим на сто порядков выше, заполняем $N_{max}+101$ и $N_{max}+100$ порядки функций нулем и единицей соотвественно.</p>
<p>В процессе расчета, следим за численными значениями функций. В тот момент, когда модуль значения расчитанной функции превысил значение $100$, проверяем порядок этой функции. Пусть порядок функции, значение модуля которой превысило значение $100$, равен $n$. Возможны два случая.</p>
<p>Если $n$ находится в диапазоне от $N_{max}$ до $N_{max} + 101$, то для оптимизации нам достаточно нормировать только $n$ и $n+1$ порядки функций делением на $100$, чтобы спускаться по индексам к требуемым порядкам, не теряя линейной связи с реальными сферическими функциями Бесселя.</p>
<p>Если же $n$ находится в диапазоне требуемых порядков, тогда требуется нормировать уже от $n$ до $N_{max}$, поэтому обратно направленным циклом переприсваиваем значения массива теми же значениями, деленными на $100$.</p>
<p>Результатом имеем полный набор сферических функций Бесселя требуемых порядков с точностью до неизвестного множителя. Вспомним, что у нас уже имеется рассчитанная часть функций Бесселя с помощью процедуры $(А1)$, найдем в ней первый попавшийся ненулевой элемент и запомним его индекс. Теперь, если мы поделим элемент нового массива того же индекса, на элемент реальных функций Бесселя, мы найдем коэффициент связи между ними, после чего сможем переприсвоить значения массива $\textit{new_sphj}$, получив, таким образом, полный набор сферических функций Бесселя. </p>

In [147]:
def upperRecurency(value, Nmax):
    
    # Nmax is the larger index of requirement functions but quantity 
    # of required functions is (Nmax + 1) cause 0-index is included
    
    # Initialization the zero-arrays for calculating the functions
    sphj, sphy, new_sphj = np.zeros(Nmax + 1), np.zeros(Nmax + 1), np.zeros(Nmax + 101)
    
    # Check for non-zero value
    if value:
        
        # Compute the spherical Bessels functions of the first kind of zero 
        # and first-indexes in the current value
        sphj[0] = sinc(value)                                                       # n = 0
        sphj[1] = (-1) * cos(value) / value + sin(value) / (value ** 2) # n = 1
        
        # Compute the spherical Bessels functions of the second kind of zero
        # and first-indexes in the current value
        sphy[0] = (-1) * cos(value) / value                                  # n = 0
        sphy[1] = (-1) * sin(value) / value - cos(value) / (value ** 2) # n = 1  
        
    
    else:
        # "value == 0" clause
        sphj[0] = special.spherical_jn(0, value)
        sphj[1] = special.spherical_jn(1, value)
        
        sphy[0] = special.spherical_yn(0, value)
        sphy[1] = special.spherical_yn(1, value)
        
    # Compute the spherical Bessels functions of the second kind another indexes
    # with bottom-to-top recurrency relation
    for ind in range(2, Nmax + 1):
        sphy[ind] = (2 * ind - 1) / value * sphy[ind - 1] - sphy[ind - 2]
        
        # For another kind of spherical Bessels functions the previous algorithm isn't stable
        # and we calculate only the part of necessary functions
        if (ind <= value):
            sphj[ind] = (2 * ind - 1) / value * sphj[ind - 1] - sphj[ind - 2]
    
    
    # For another part we should use the top-to-bottom recurrency relation
    
    # Initialization {Nmax + 101} and {Nmax + 100}-indexes for this relation.
    # We should extend our quantity of calculating functions by 100
    # After that our 'new' spherical Bessels functions of the first kind will be known,
    # but it will be the real functions multiplied by the some const.
    new_sphj[Nmax + 100] = 0 # n = Nmax + 101
    new_sphj[Nmax + 99] = 1 # n = Nmax + 100
    
    # Calculate the 'new' spherical Bessels functions of the first kind of {98 + Nmax} down to {0} - indexes
    for ind in range(98 + Nmax, -1, -1):
        new_sphj[ind] = (2 * ind + 3) / value * new_sphj[ind + 1] - new_sphj[ind + 2]
        
        # We shouldn't forget about the fact that our 'new' functions dramatically increases step-by-step.
        # To avoid overflowing, we reduce all of the previous caltulating functions by division on 100
        if abs(new_sphj[ind]) > 100:
            
            # We don't care about indexes which we don't want to know
            if ind <= Nmax:
                for index in range(ind, Nmax, 1):
                    new_sphj[index] /= 100
                    
            # For this case we should reduce only the {current index + 1} and {current index} functions
            # to continue our calculation
            else:
                new_sphj[ind + 1] /= 100
                new_sphj[ind] /= 100
                
    # After all calculations we want to know the const of relation between the 'new'
    # and known Bessels functions of the first kind
    for ind in range(len(sphj)):
        
        # Check for zero division to avoid the problem
        if sphj[ind] != 0:
            k = new_sphj[ind] / sphj[ind]
            break
    
    # Initialize the new array for recalculation our 'new' Bessels functions with known constant
    bessels = []
    
    # Division all the 'new' functions by known constant
    for elem in new_sphj:
        bessels.append(elem / k)
#         print(elem / k)
    
#     # Check the top-to-bottom and bottom-to-top recurrency relations
#     for ind in range(len(sphj)):
#         if sphj[ind]:
#             print(f'top_to_bottom, j_{ind}({value}) = {bessels[ind]}\n', f'bottom_to_top, j_{ind}({value}) = {sphj[ind]}\n')

    # After all kinds of calculations, we return the 1st and 2nd kind of functions as 2 arrays in the current value
    return bessels[:Nmax + 1], sphy
    
# print(upperRecurency(20, 10))

<p>Рассмотрим подробнее результат функции $\textit{upperRecurency}$.</p> 
<p>Для этого вызовем эту функцию, для аргументов $N_{max} = 10$, $\text{value} = 20$ и сохраним их в одноименные с названием сферических функций переменные.</p>

In [148]:
value = 20
n_max = 10
bessels, neymans = upperRecurency(value, n_max)

<p>Напечатаем, что представляют из себя массивы в случае заданных параметров.</p>

In [149]:
for ind in range(len(bessels)):
    print(f'j_{ind}({value}) = {format(bessels[ind], ".5f")}')

j_0(20) = 0.04565
j_1(20) = -0.01812
j_2(20) = -0.04837
j_3(20) = 0.00603
j_4(20) = 0.05048
j_5(20) = 0.01668
j_6(20) = -0.04130
j_7(20) = -0.04353
j_8(20) = 0.00865
j_9(20) = 0.05088
j_10(20) = 0.03969


In [150]:
for ind in range(len(neymans)):
    print(f'y_{ind}({value}) = {format(neymans[ind], ".5f")}')

y_0(20) = -0.02040
y_1(20) = -0.04667
y_2(20) = 0.01340
y_3(20) = 0.05002
y_4(20) = 0.00410
y_5(20) = -0.04817
y_6(20) = -0.03060
y_7(20) = 0.02828
y_8(20) = 0.05181
y_9(20) = 0.01575
y_10(20) = -0.03684


<p>Особого смысла здесь нет, но можно убедиться, что функция работает и позволяет получать требуемые сферические функции.</p>

<p>Определим теперь функцию $\textit{derivative}$, которая будет получать на вход набор уже известных нам сферических функций Неймана и Бесселя, и их аргумент. Далее реализуем процедуру $(A2)$, для этого, инициализируем начальные значения, и производим рекуррентный расчет. Результатом функции является кортеж одномерных массивов производных от функций Бесселя и Неймана с аргументом, переданным в эту функцию. </p>

In [151]:
def derivative(sphj, sphy, value):
    # Initialization the zero-arrays for calculating the derivative functions
    sphj_der, sphy_der = np.zeros(len(sphy)), np.zeros(len(sphy))
    
    # Define ititial derivatives
    sphj_der[0] = cos(value) / value - sin(value) / (value ** 2)
    sphy_der[0] = sin(value) / value + cos(value) / (value ** 2)
    
    # Compute another indexes with recurrency relation for derivatives
    for ind in range(1, len(sphy)):
        sphj_der[ind] = sphj[ind - 1] - (ind + 1) / value * sphj[ind]
        sphy_der[ind] = sphy[ind - 1] - (ind + 1) / value * sphy[ind]
    return sphj_der, sphy_der

<p>Убедимся, что и эта функция работает, используем все те же параметры, что и выше</p>

In [152]:
bessels_der, neymans_der = derivative(bessels, neymans, value)

Напечатаем, что представляют из себя массивы производных

In [153]:
for ind in range(len(bessels_der)):
    print(f'j_{ind}_der({value}) = {format(bessels_der[ind], ".5f")}')

j_0_der(20) = 0.01812
j_1_der(20) = 0.04746
j_2_der(20) = -0.01087
j_3_der(20) = -0.04957
j_4_der(20) = -0.00659
j_5_der(20) = 0.04547
j_6_der(20) = 0.03114
j_7_der(20) = -0.02389
j_8_der(20) = -0.04742
j_9_der(20) = -0.01679
j_10_der(20) = 0.02906


In [154]:
for ind in range(len(neymans_der)):
    print(f'y_{ind}_der({value}) = {format(neymans_der[ind], ".5f")}')

y_0_der(20) = 0.04667
y_1_der(20) = -0.01574
y_2_der(20) = -0.04868
y_3_der(20) = 0.00340
y_4_der(20) = 0.04899
y_5_der(20) = 0.01855
y_6_der(20) = -0.03746
y_7_der(20) = -0.04191
y_8_der(20) = 0.00497
y_9_der(20) = 0.04393
y_10_der(20) = 0.03602


<p>Теперь вернемся к тому, зачем, собственно, и создавались все выше перечисленные функции. Нам хотелось бы получить численное выражение радиационной силы, действующей на некую упругую сферическую частицу(рассеиватель), размещенную в начале координат. При определении радиационной силы на некоторое препятствие следует учитывать изменение импульса волны из-за рассеяния волны на нем [3 – 5]. В связи с этим алгоритм расчета состоит из двух основных этапов: сначала решается задача о рассеянии звука на сфере, а затем полученные данные для рассеянной волны используются для расчета радиационной силы.</p>
<p>Первым этапом расчетов является решение задачи рассеяния. В качестве рассеивателя для начала будем рассматривать абсолютно жесткую сферу.</p>
<p>Будем полагать, что окружающая рассеиватель среда является идеальной жидкостью
с плотностью $\rho$ и скоростью распространения звука в ней $с$. Поместим в жидкость
абсолютно жесткую сферу радиуса а и введем сферическую систему координат с началом в центре сферы (рис. 1.1).</p>

<img src=https://bit.ly/2zRSPp8 width="500">

$$\textbf{Рис. 1.1.} \text{Геометрия задачи рассеяния плоской волны на сфере}$$

<p>Пусть на сферу (рис. 1.1.) падает плоская гармоническая волна. В этом случае акустическое давление $p'(\vec{r},t)$ и колебательная скорость частиц $\vec{v'}(\vec{r},t)$ запишутся:
    
$$p' = \frac{P}{2} e^{-i\omega t} + \frac{P^*}{2}e^{i\omega t}, \;\; \vec{v'} = \frac{\vec{V}}{2}e^{-i\omega t} + \frac{\vec{V^*}}{2}e^{i\omega t} , \tag{1.1}$$</p>
 
<p>где $P$, $\vec{V}$ - комплексные амплитуды волны, $f = \omega / (2\pi)$ - ее частота.</p>
 
<p>Комплексная амплитуда $P$ является решением уравнения Гельмгольца $\Delta P +k^2P = 0$, где $k = \omega / c$ -- волновое число. Ее можно представить в виде суммы комплексных амплитуд падающей и рассеянной волн: $P = P_\text{пад} + P_\text{расс}$</p>

<p>В случае плоской падающей волны величина $P_\text{пад}$ представима в виде следующего разложения [1]:</p>

$$P_\text{пад} = p_0e^{ikrcos\theta} = p_0\sum\limits_{n=0}^{\infty} i^n(2n+1)j_n(kr)P_n(cos\theta), \tag{1.2}$$

<p>где $r$ и $\theta$ -- сферические координаты (рис. 1.1), $j_n(\zeta) = \sqrt{\pi/(2\zeta)}J_{n+1/2}(\zeta)$ - сферическая функция Бесселя, $P_n(cos\theta)$ -- полином Лежандра. Рассеянное поле представимо в виде похожего разложения [2]:</p>

$$P_\text{расс}=p_0\sum\limits_{n=0}^{\infty}c_nh_n^{(1)}(kr)P_n(cos\theta), \tag{1.3}$$

<p>где $h_n^{(1)}(kr) =j_n(kr) +iy_n(kr)$ - функция Ханкеля 1-го рода, $y_n(\zeta) = \sqrt{\pi/(2\zeta)}Y_{n+1/2}(\zeta)$, $Y_n(\zeta)$ - функция Неймана. Коэффициенты $c_n$ находятся из граничных условий на поверхности рассеивателя. Поскольку жидкость считается невязкой, указанные условия заключаются в непрерывности нормальных компонент скорости и напряжений, а также отсутствии касательного напряжения на поверхности рассеивателя. В частном случае абсолютно жесткого рассеивателя граничыне условия упрощаются и сводятся к одному условию равенства нулю нормальной компоненты скорости $V_n = 0$. В случае сферы нормальной компонентой скорости является ее радиальная компонента $V_r=0$.</p>

<p>Граничное условие при $r = a$ запишется так:</p>

$$V_r = \frac{1}{i\omega \rho}\frac{\partial{P}}{\partial{r}}\Bigl|_{r=a} \tag{1.4}$$

<p>Используя соотношения (1.2), (1.3), (1.4) получим:</p>

$$c_n = -i^n(2n+1)\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)}. \tag{1.5} $$

<p>Здесь штрих означает производную по аргументу соотвествтующей фукнкции. Из (1.1) --- (1.5) получим, что амплитуда полного поля равна:</p>

$$P = p_0\sum\limits_{n=0}^{\infty}i^n(2n+1)\Bigl[j_n(kr) -\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)} h_n^{(1)}(kr)\Bigr]P_n(cos\theta). \tag{1.6}$$

<p>Аналогично найдем выражения для комплексных амплитуд радиальной $V_r$ и угловой $V_\theta$ компонент:</p>

$$V_r =\frac{1}{i\rho c}p_0\sum\limits_{n=0}^{\infty}P_n(cos\theta)(2n+1)i^n\Bigl[j_n(kr) -\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)} h_n^{(1)}(kr)\Bigr] \tag{1.7}$$

$$V_\theta = \frac{1}{i\rho \omega}\frac{1}{r}p_0\sum\limits_{n=0}^{\infty} \frac{\partial{}}{\partial\theta} (P_n(cos\theta))(2n+1)i^n\Bigl[j_n(kr) -\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)} h_n^{(1)}(kr)\Bigr] \tag{1.8}$$

<p>Выражения (1.6), (1.7), (1.8) полностью описывают поле звуковой волны, рассеиваемой на абсолютно жесткой сфере радиуса $a$.</p>

<p>На основе описанной теории, приходим к выводу о том, что мы не можем реализовать бесконечное суммирование ряда (см. формулу $(1.2)$), поэтому нам нужно определить верхний предел суммирования, который бы смог гарантировать корректность всех рассчитываемых сумм. Таким образом, сначала решался вопрос поиска таких значений верхней границы суммирования $N_{max}$ данного ряда, для которых отбрасываемая часть суммы вносит незначительный вклад</p>

<p>На помощь приходит представление или графическая интерпретация сферических функций Бесселя (рис. 1.2.), которые согласно формуле $(1.2)$ входят в искомый ряд.</p>

<img src=https://bit.ly/3b3lHYs width="500">

$$\textbf{Рис. 1.2.} \text{Сферические функции Бесселя}$$

<p>Нетрудно заметить, что по мере увеличения порядка сферической функции, ее первый экстремум сдвигается вправо вдоль оси аргумента, а все, что находится левее первого экстремума, характеризуется малыми значениями. Соответственно, если зафиксировать произвольное значение аргумента, то наибольший вклад в искомый ряд будет вносить сферическая функция Бесселя наименьшего порядка, а каждая последующая функция  -- все меньше и меньше, и, начиная с какого-то номера $N_{max}$, вклад функций с порядком большим, чем $N_{max}$, будет уже пренебрежимо мал для построения нашей модели.</p>
	
<p>Согласно условиям данной задачи нам требуется исследовать влияние падающего поля давления на рассеиватель, поэтому необходимо реализовать точный расчет лишь вблизи локализации рассеивателя.</p>
	
<p>Учитывая вышеизложенные факты, получаем, что при аргументе функции, которым является величина $kr$ согласно $(1.2)$, равном $N_{max}$, последующие сферические функции Бесселя порядков выше, чем $N_{max}$, не вносят существенный вклад в рассчитываемый ряд, а сам радиус области оказывается равным радиусу рассеивателя (назовем его $a$), умноженному на число из промежутка от трех до пяти.</p>
	
<p>Таким образом, мы получили выражение для предельных значений верхней границы суммирования, при которых расчитанное поле давлений сходится достаточно точно с теорией вблизи области $r = a$.</p>

$$N_{max} = (3\div5)ka. \tag{1.12}$$

<p>Получается, выбирая значение $N_{max}$, мы фактически задаем максимально возможные значения радиуса нашего рассеивателя. Если же радиус рассеивателя положить фиксированным, а значение верхней границы суммирования взять меньшим, чем значение, полученное в результате подстановки соответсвующего радиуса в уравнение $(1.12)$, мы можем столкнуться с неправильным подсчетом сферических функций в этой области.</p>

<p>В случае сферической падающей волны, согласно [6] существует похожее относительно $(1.2)$ разложение:</p>

$$P_s = \sum \limits_{n=0}^{\infty} A_{sp} ik(-1)^n(2n+1)h_n^{(1)}(kz_0)c_nh_n^{(1)}(kr)P_n(cos\theta), \tag{1.13}$$

<p>где $c_n$ определяется согласно формуле (1.3), а $h_n^{(1)}(kr) =j_n(kr) +iy_n(kr)$ - функция Ханкеля 1-го рода, $y_n(\zeta) = \sqrt{\pi/(2\zeta)}Y_{n+1/2}(\zeta)$, $Y_n(\zeta)$ - функция Неймана. Источник сферической волны находится в точке $z = -z_0$.</p>

<p>В случае сферической падающей волны величина $P_{\text{пад}}$ представима в виде:</p>

$$P_{\text{пад}} = p_0e^{ikR}/R, \tag{1.14}$$

<p>где $R = \sqrt{r^2 + z_0^2+2rz_0cos\theta}~~$(рис. 1.5).</p>

<img src=https://bit.ly/2WpuPS5 width="500">

$$\textbf{Рис. 1.3.}~ \text{Геометрия задачи в случае сферической волны}$$

<h2>Рассмотрим теперь основную программу</h2>

<p>Для лучшей читаемости заменим $R$ из рис. 1.2 на функцию $\textit{sqrthcos}$, название ее связано с теоремой косинусов, применяемой для вычисления самой величины. На вход этой функции передаем радиус-вектор рассматриваемой точки, координату источника сферической волны, и косинус угла до радиус-вектора этой точки, для однозначности. На выходе получаем значение $R$.</p>

<p>Инициализация начальных параметров, констант, ничего особенного, вызов функции $\textit{upperRecurency}$ для рассчета функций Бесселя и Неймана в точке с координатой источника волн, которые потом используются в формуле $(1.13)$</p>

In [172]:
# Square root + theorem of cos
def sqrthcos(r, z0, costheta):
    return (r ** 2 + z0 ** 2 + 2 * r * z0 * costheta) ** .5

# Initialize the parameters of required functions
# Nmax -> quantity of required functionsp
# z0 -> initial coordinate of the spherical-wave sourse
# step -> cells-step
Nmax = 30
z0 = 50
step = 1
a = 5 * (10 ** (-3))  # 5mm radius of sperical object
f = 10 ** 6  # 1 MHz
# k = 2 * pi * f / 1500  # k = w / c, w = 2piv
k = 1
p_0 = 1
A_sp = 1
rho = 1
c_sound = 1 #1500


# Initialize the matrix 401 x 401 ([-200, 200] x [-200, 200]) with complex data type for theoretical functions
p_ith = np.zeros((401, 401), dtype=complex)
p_spth = np.zeros((401, 401), dtype=complex)
p_sc = np.zeros((401, 401), dtype=complex)
p_sc_sp = np.zeros((401, 401), dtype=complex)


# Initialize the matrix 1 x Nmax for Legendre Polynomials
P = np.zeros(Nmax + 1)

# Initialize the matrix 401 x 401 ([-200, 200] x [-200, 200]) with complex data type for calculating functions
p_i = np.zeros((401, 401), dtype=complex)
p_sp = np.zeros((401, 401), dtype=complex)

# Calculating the Bessels functions with the help of our function called as 'upperRecurency' in the value z0
sphjz0, sphyz0 = upperRecurency(k * z0, Nmax)

t = time.time()
e = np.zeros((401, 401), dtype=float)
e_sp = np.zeros((401, 401), dtype=float)

<p>Для лучшего восприятия введем вспомогательные функции для сферического коэффициента $Q_n^{(spherical~wave)}$ и коэффициента плоской волны $Q_n^{(plane~wave)}$ согласно [6].</p>

<p>Формульно они записываются так:</p>

$$Q_n^{(spherical~wave)} = A_{sp} ik(-1)^n(2n+1)h_n^{(1)}(kz_0) \tag{1.15}$$

$$Q_n^{(plane~wave)} = p_0i^n(2n+1)\tag{1.16}$$

In [173]:
def Q_sp(n, sphjz0, sphyz0):
    return A_sp * 1j * k * ((-1) ** n) * (2 * n + 1) * (sphjz0[n] + 1j * sphyz0[n])

In [174]:
def Q_pl(n):
    return p_0 * (1j ** n) * (2 * n + 1)

<p>Вычислим коэффициенты $c_n$ в разложении $(1.13)$ используя $(1.5)$.</p>

<p>Радиационная сила, точнее ее проекция на ось $z$, может быть представлена как:</p>

$$F_z = \frac{2\pi}{\rho c^2 k^2}\sum_{n=0}^{\infty}\frac{(n+1)}{(2n+1)(2n+3)}Im[Q_nQ_{n+1}^*(c_n+c_{n+1}^*+2c_nc_{n+1}^*)]\tag{1.17}$$

<p>Таким образом, правильно вычислив сферические функции Бесселя и Неймана и их производные, мы можем вычислить коэффициенты $c_n$, с помощью которых получаем искомое выражения для проекции радиационной силы</p>

In [175]:
# calculate the coefficients in the sum for spherical wave
c = np.zeros(Nmax + 1, dtype=complex)
F_z_plane, F_z_spherical = 0, 0
sphja, sphya = upperRecurency(k * a, Nmax)
sphja_der, sphya_der = derivative(sphja, sphya, k * a)
# print(sphja_der,sphya_der)
for n in range(len(sphya)):
    c[n] = -(1j) ** n * (2 * n + 1) * sphja_der[n] / (sphja_der[n] + 1j * sphya_der[n]) 


for n in range(Nmax):
    F_z_spherical += 2 * pi / (rho * (c_sound ** 2) * (k ** 2)) *(n + 1) / ((2 * n + 1) * (2 * n + 3)) * np.imag(Q_sp(n, sphjz0, sphyz0) * np.conj(Q_sp(n + 1, sphjz0, sphyz0)) * (c[n] + np.conj(c[n + 1]) + 2 * c[n] * np.conj(c[n + 1])))
    F_z_plane += -2 * pi * (p_0 ** 2) / (rho * (c_sound ** 2) * (k ** 2)) * (n + 1) * np.real(c[n] + np.conj(c[n + 1]) + 2 * c[n] * np.conj(c[n + 1]))

<p>Посмотрим, чему равны значения рассчитанных радиационных сил, в ознакомительных целях</p>

In [176]:
print(f'F_z_spherical_wave = {F_z_spherical}')

F_z_spherical_wave = 4.733297900127387e-10


In [177]:
print(f'F_z_plane_wave = {F_z_plane}')

F_z_plane_wave = 1.1780884201844693e-06


<p>Следующий блок кода был положен в основу проверки корректности расчета сферических функций. С помощью формул $(1.2)$ и $(1.14)$ мы знаем математическое описание поля давлений в заданной области, поэтому, рассчитав частичные суммы рядов $(1.2)$ и $(1.13)$, мы можем изобразить модуль разности теоретических значений и рассчитанных, оценив, при этом, область сходимости, то есть ту область, в которой наши функции посчитаны верно.</p>

In [178]:
# Going through the cells
for x in range(-200, 201):
    print(x / 4 + 50, '%', '\r', end='')
    for z in range(-200, 201):
        
        # Calculating the radius-vector in the current point
        r = step * (x ** 2 + z ** 2) ** .5
        
        # Check for non-zero value for r
        if r:
            
            # Calculating cos(theta) in the current point
            costheta = step * z / r
            
                
            # Calculating the Bessels functions with the help of our function called as 'upperRecurency' in the value r
            sphj, sphy = upperRecurency(k * r, Nmax)
                
            # Calculating the theoretical pressure functions
            p_ith[x + 200][z + 200] = p_0 * cmath.exp(1j * k * r * costheta)
                
            # Check for non-zero value for theorem of cos in the current point
            if sqrthcos(r, z0, costheta):
                p_spth[x + 200][z + 200] = A_sp * cmath.exp(1j * k * sqrthcos(r, z0, costheta)) / sqrthcos(r, z0, costheta)
                
            # Use bottom-to-top recurrency relation to calculate Legendre's polynomials
            P[0] = 1
            P[1] = costheta
            for n in range(1, Nmax):
                P[n + 1] = (2 * n + 1) / (n + 1) * costheta * P[n] - n / (n + 1) * P[n - 1];
                    
            # Calculating the required pressure functions (incident)
            for n in range(0, Nmax):
                p_i[x + 200, z + 200] += Q_pl(n) * P[n] * sphj[n];
                p_sp[x + 200, z + 200] += Q_sp(n, sphjz0, sphyz0) * P[n] * sphj[n] 
                
                # (scattered)
                p_sc [x + 200, z + 200] += Q_pl(n) * c[n] * (sphj[n] + 1j * sphy[n]) * P[n]
                p_sc_sp [x + 200, z + 200] += Q_sp(n, sphjz0, sphyz0) * c[n] * (sphj[n] + 1j * sphy[n]) * P[n]
                
            e[x + 200, z + 200] = abs(p_i[x + 200, z + 200] - p_ith[x + 200, z + 200])
            e_sp[x + 200, z + 200] = abs(p_sp[x + 200, z + 200] - p_spth[x + 200, z + 200])
                
print(time.time() - t)

183.9994580745697%  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  %  % 


<p>Картинки, на которых можно оценить, как влияет $N_{max}$ на размеры области сходимости</p>

In [179]:
Image(url='https://bit.ly/3c6Ix2D')

$$\textbf{рис. 1.4.} ~~\text{$N_{max} = 10~$Плоская волна}$$

In [183]:
Image(url='https://bit.ly/2SAyxao')

$$\textbf{рис. 1.5.} ~~\text{$N_{max} = 10~$Сферическая волна}$$

In [184]:
Image(url='https://bit.ly/2Wpzxzi')

$$\textbf{рис. 1.6.} ~~\text{$N_{max} = 30~$Плоская волна}$$

In [187]:
Image(url='https://bit.ly/2L1NCxm')

$$\textbf{рис. 1.7.} ~~\text{$N_{max} = 30~$Сферическая волна}$$

<p>Здесь ниже ничего интересного, формирование картинок, но так как вы их покрутить не сможете, я их прикрепил выше.</p>

In [180]:
x = np.arange(-200, 201, 1)
y = np.arange(-200, 201, 1)

z = np.zeros((401, 401), dtype=float)
for ind in range(len(e)):
    for sec_ind in range(len(e[ind])):
        z[ind][sec_ind] = e[ind][sec_ind]
        
z2 = np.zeros((401, 401), dtype=float)
for ind in range(len(e_sp)):
    for sec_ind in range(len(e_sp[ind])):
        z2[ind][sec_ind] = e_sp[ind][sec_ind]


for ind in range(len(z)):
    for sec_ind in range(len(z[ind])):
#         print(z[ind][sec_ind] - e[ind][sec_ind])
        if z[ind][sec_ind] > 2:
#             print(p_i[ind][sec_ind], p_ith[ind][sec_ind])
            z[ind][sec_ind] = 2
    
for ind in range(len(z2)):
    for sec_ind in range(len(z2[ind])):
#         print(z[ind][sec_ind] - e[ind][sec_ind])
        if z2[ind][sec_ind] > .1:
#             print(p_i[ind][sec_ind], p_ith[ind][sec_ind])
            z2[ind][sec_ind] = .1

In [188]:
# fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
# fig.update_layout(title='Error Function Plane Wave', autosize=False,
#                   width=800, height=800,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

In [189]:
# fig = go.Figure(data=[go.Surface(z=z2, x=x, y=y)])
# fig.update_layout(title='Error Function Spherical Wave', autosize=False,
#                   width=800, height=800,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

<ul>Список Литературы:</ul>
<p>1) $\textit{Hasegawa T., Yosioka K.}$ Acoustic-radiation force on a solid elastic sphere // J. Acoust. Soc. Amer. 1969. \\ V. 46. №. 5B. P. 1139 - 1143.</p>
<p>2) $\textit{Морс Ф. М., Фешбах Г.}$ Методы теоретической физики. М.: ИЛ, 1960. Т. 2.</p>
<p>3) $\textit{Lee C. P., Wang T. G.}$ Acoustic radiation pressure // J. Acoust. Soc. Amer. 1993. V. 94. No.
 2. P. 1099 – 1109.</p>
<p>4) $\textit{Hasegawa T. Kido T., Iizuka T., Matsuoka C.}$ A general theory of Rayleigh and Langevin radiation pressures // Acoust. Science Techn. 2001. V. 21. No. 3. P. 145 – 152.</p>
<p>5) $\textit{Ландау Л.Д., Лифшиц Е.М.}$ Гидродинамика, Ч. 1. M.: Наука, 1987, Глава 1. Идеальные среды. С. 1 – 43.</p>
<p>6) $\textit{Sapozhnikov O. A., Bailey M. R.}$ Radiation force on an elastic sphere //J. Acoust. Soc. Am. 2013  Vol. 133. №. 2. P. 661 - 676.</p>
<p>7)$~$<a>https://en.wikipedia.org/wiki/Bessel_function#Spherical_Bessel_functions</a></p>
<p>8)$~$<a>https://docs.scipy.org/doc/scipy/reference/special.html</a></p>
<p>9)$~$<a>https://plotly.com/python/</a></p>